In [1]:
!nvidia-smi
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

/bin/bash: nvidia-smi: command not found
--2022-02-13 09:44:17--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-02-13 09:44:17--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-02-13 09:44:17--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [applicatio

In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
train_df = pd.read_csv('mami_training.csv', sep='\t', lineterminator='\r')
test_df = pd.read_csv('Test.csv', sep='\t', lineterminator='\n')
test_labels = pd.read_csv('test_labels.txt',header=None, sep='\t', lineterminator='\n')
X = train_df['Text Transcription'].copy()
for i in range(len(X)):
  X[i] = X[i].lower()
X_test_model = test_df['Text Transcription'].to_numpy()
for i in range(len(X_test_model)):
  X_test_model[i] = X_test_model[i].lower()
Y = train_df['misogynous']
X_train_model = X[:].to_numpy()
Y_train_model = Y[:].to_numpy()
print(X_train_model.shape, Y_train_model.shape)
print(Y_train_model.sum())

(10000,) (10000,)
5000


In [3]:
test_labels[1].sum()

500

In [4]:
train_df.head()

,file_name,misogynous,shaming,stereotype,objectification,violence,Text Transcription
0,\n1.jpg,0,0,0,0,0,Milk Milk.zip
1,\n10.jpg,1,0,0,0,1,"ROSES ARE RED, VIOLETS ARE BLUE IF YOU DON'T S..."
2,\n1000.jpg,0,0,0,0,0,BREAKING NEWS: Russia releases photo of DONALD...
3,\n10000.jpg,0,0,0,0,0,MAN SEEKING WOMAN Ignad 18 O
4,\n10006.jpg,0,0,0,0,0,Me explaining the deep lore of. J.R.R. Tolkein...


In [5]:
embeddings_index = {}

f = open('glove.6B.200d.txt')
for line in f:
    values = line.split()
    word = value = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' %len(embeddings_index))

Found 400000 word vectors.


In [6]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence  import pad_sequences

tokenizer_obj = Tokenizer()
total_memes =  train_df['Text Transcription']
tokenizer_obj.fit_on_texts(total_memes)

max_length = 50 #max([len(s.split()) for s in total_memes])
vocab_size  = len(tokenizer_obj.word_index) + 1
X_train_tokens = tokenizer_obj.texts_to_sequences(X_train_model)
X_test_tokens = tokenizer_obj.texts_to_sequences(X_test_model)
X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length)
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length)
print(vocab_size)

20693


In [7]:
EMBEDDING_DIM = 200
word_index = tokenizer_obj.word_index
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector

def average_glove(df):
  real_X = np.zeros((len(df) , EMBEDDING_DIM))
  for i in range(len(df)):
    num = 0
    for word in df[i]:
      num += 1
      real_X[i] += embedding_matrix[int(word)]
    if num!=0:
      real_X[i] = real_X[i]/num
  return real_X

X_train_pad_2_vec = average_glove(X_train_pad)
X_test_pad_2_vec = average_glove(X_test_pad)
print(X_test_pad_2_vec.shape,X_train_pad_2_vec.shape )

(1000, 200) (10000, 200)


In [8]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import confusion_matrix
Y_test_model = test_labels[1].to_numpy()
Y_train_model = train_df['misogynous'].to_numpy()
lr_clf = LogisticRegressionCV(random_state=42, solver='liblinear', max_iter=1000, n_jobs=5, verbose=2, scoring='f1_macro', refit=True)
lr_clf.fit(X_train_pad_2_vec[:8000], Y_train_model[:8000])
print("######## Misogynous Prediction ########")
print("\nTraining Dataset ")
y_pred = lr_clf.predict(X_train_pad_2_vec[:8000])
print(confusion_matrix(Y_train_model[:8000], y_pred),"\n F1 score - ", f1_score(Y_train_model[:8000], y_pred, average='macro'))
print("\nValidation Dataset ")
y_pred = lr_clf.predict(X_train_pad_2_vec[8000:])
print(confusion_matrix(Y_train_model[8000:], y_pred),"\n F1 score - ", f1_score(Y_train_model[8000:], y_pred, average='macro'))
y_pred = lr_clf.predict(X_test_pad_2_vec)
print("\nSubmission Score ")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskA = f1_score(Y_test_model, y_pred, average='macro')
subtaskB = f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:   23.0s remaining:   34.4s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:   23.8s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:   23.8s finished


[LibLinear]######## Misogynous Prediction ########

Training Dataset 
[[2887 1076]
 [1007 3030]] 
 F1 score -  0.7395417795288998

Validation Dataset 
[[734 303]
 [276 687]] 
 F1 score -  0.7103400352844859

Submission Score 
[[224 276]
 [128 372]] 
 F1 score -  0.5869526099687556


In [9]:
Y_train_model = train_df['shaming'].to_numpy()
Y_test_model = test_labels[2].to_numpy()
lr_clf = LogisticRegressionCV(random_state=42, solver='liblinear', max_iter=1000, n_jobs=5, verbose=2, scoring='f1_macro', refit=True)
lr_clf.fit(X_train_pad_2_vec[:8000], Y_train_model[:8000])
print("######## Shaming Prediction ########")
print("\nTraining Dataset ")
y_pred = lr_clf.predict(X_train_pad_2_vec[:8000])
print(confusion_matrix(Y_train_model[:8000], y_pred),"\n F1 score - ", f1_score(Y_train_model[:8000], y_pred, average='macro'))
print("\nValidation Dataset ")
y_pred = lr_clf.predict(X_train_pad_2_vec[8000:])
print(confusion_matrix(Y_train_model[8000:], y_pred),"\n F1 score - ", f1_score(Y_train_model[8000:], y_pred, average='macro'))
y_pred = lr_clf.predict(X_test_pad_2_vec)
print("\n######## Shaming Prediction ########")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskB += f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:   16.7s remaining:   25.1s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:   17.3s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:   17.3s finished


[LibLinear]######## Shaming Prediction ########

Training Dataset 
[[6868   76]
 [ 917  139]] 
 F1 score -  0.5756536970931263

Validation Dataset 
[[1763   19]
 [ 190   28]] 
 F1 score -  0.5776817963678614

######## Shaming Prediction ########
[[838  16]
 [133  13]] 
 F1 score -  0.5334637964774951


In [10]:
Y_train_model = train_df['stereotype'].to_numpy()
Y_test_model = test_labels[3].to_numpy()
lr_clf = LogisticRegressionCV(random_state=42, solver='liblinear', max_iter=1000, n_jobs=5, verbose=2, scoring='f1_macro', refit=True)
lr_clf.fit(X_train_pad_2_vec[:8000], Y_train_model[:8000])
print("######## Stereotype Prediction ########")
print("\nTraining Dataset ")
y_pred = lr_clf.predict(X_train_pad_2_vec[:8000])
print(confusion_matrix(Y_train_model[:8000], y_pred),"\n F1 score - ", f1_score(Y_train_model[:8000], y_pred, average='macro'))
print("\nValidation Dataset ")
y_pred = lr_clf.predict(X_train_pad_2_vec[8000:])
print(confusion_matrix(Y_train_model[8000:], y_pred),"\n F1 score - ", f1_score(Y_train_model[8000:], y_pred, average='macro'))
y_pred = lr_clf.predict(X_test_pad_2_vec)
print("\n######## Stereotype Prediction ########")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskB += f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:   19.6s remaining:   29.3s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:   20.1s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:   20.1s finished


[LibLinear]######## Stereotype Prediction ########

Training Dataset 
[[5142  442]
 [1546  870]] 
 F1 score -  0.6523717062743759

Validation Dataset 
[[1442  164]
 [ 247  147]] 
 F1 score -  0.646143415232622

######## Stereotype Prediction ########
[[575  75]
 [263  87]] 
 F1 score -  0.5563466061827956


In [11]:
Y_train_model = train_df['objectification'].to_numpy()
Y_test_model = test_labels[4].to_numpy()
lr_clf = LogisticRegressionCV(random_state=42, solver='liblinear', max_iter=1000, n_jobs=5, verbose=2, scoring='f1_macro', refit=True)
lr_clf.fit(X_train_pad_2_vec[:8000], Y_train_model[:8000])
print("######## Objectification Prediction ########")
print("\nTraining Dataset ")
y_pred = lr_clf.predict(X_train_pad_2_vec[:8000])
print(confusion_matrix(Y_train_model[:8000], y_pred),"\n F1 score - ", f1_score(Y_train_model[:8000], y_pred, average='macro'))
print("\nValidation Dataset ")
y_pred = lr_clf.predict(X_train_pad_2_vec[8000:])
print(confusion_matrix(Y_train_model[8000:], y_pred),"\n F1 score - ", f1_score(Y_train_model[8000:], y_pred, average='macro'))
y_pred = lr_clf.predict(X_test_pad_2_vec)
print("\n######## Objectification Prediction ########")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskB += f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:   20.4s remaining:   30.6s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:   21.3s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:   21.3s finished


[LibLinear]######## Objectification Prediction ########

Training Dataset 
[[5949  218]
 [1457  376]] 
 F1 score -  0.5932203924931434

Validation Dataset 
[[1567   64]
 [ 298   71]] 
 F1 score -  0.5890995604954414

######## Objectification Prediction ########
[[609  43]
 [293  55]] 
 F1 score -  0.5152102775421161


In [12]:
Y_train_model = train_df['violence'].to_numpy()
Y_test_model = test_labels[5].to_numpy()
lr_clf = LogisticRegressionCV(random_state=42, solver='liblinear', max_iter=1000, n_jobs=5, verbose=2, scoring='f1_macro', refit=True)
lr_clf.fit(X_train_pad_2_vec[:8000], Y_train_model[:8000])
print("######## Violence Prediction ########")
print("\nTraining Dataset ")
y_pred = lr_clf.predict(X_train_pad_2_vec[:8000])
print(confusion_matrix(Y_train_model[:8000], y_pred),"\n F1 score - ", f1_score(Y_train_model[:8000], y_pred, average='macro'))
print("\nValidation Dataset ")
y_pred = lr_clf.predict(X_train_pad_2_vec[8000:])
print(confusion_matrix(Y_train_model[8000:], y_pred),"\n F1 score - ", f1_score(Y_train_model[8000:], y_pred, average='macro'))
y_pred = lr_clf.predict(X_test_pad_2_vec)
print("\n######## Violence Prediction ########")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskB += f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:   17.7s remaining:   26.5s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:   18.1s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:   18.1s finished


[LibLinear]######## Violence Prediction ########

Training Dataset 
[[7133   36]
 [ 755   76]] 
 F1 score -  0.5543269967900698

Validation Dataset 
[[1860   18]
 [ 114    8]] 
 F1 score -  0.5369200976677613

######## Violence Prediction ########
[[836  11]
 [141  12]] 
 F1 score -  0.5265151515151516


In [13]:
print("Subtask A score -", 
      subtaskA, 
      "\n Subtask B score - ", 
      subtaskB/(test_labels[5].sum()+ test_labels[4].sum()+ test_labels[3].sum()+test_labels[2].sum()+test_labels[1].sum()))

Subtask A score - 0.5869526099687556 
 Subtask B score -  0.5517256688046395
